Import Dependencies

In [1]:
import openai
import instructor
from qdrant_client import QdrantClient
from pydantic import BaseModel, Field

RAG Pipeline

In [2]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")

In [3]:
client = instructor.from_openai(openai.OpenAI())

In [4]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature = 0.0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "data_model": answer,
        "answer": answer.answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [5]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [6]:
output = rag_pipeline("What is the most popular product?", qdrant_client)

In [7]:
output

{'data_model': RAGGenerationResponse(answer='The most popular product based on the provided ratings is the 1TB USB Flash Drive (ID: B0CF57H28T) with a rating of 4.8.'),
 'answer': 'The most popular product based on the provided ratings is the 1TB USB Flash Drive (ID: B0CF57H28T) with a rating of 4.8.',
 'question': 'What is the most popular product?',
 'retrieved_context_ids': ['B0CH6P8DYF',
  'B0BFPZGYLD',
  'B0CF57H28T',
  'B09LVX3XW2',
  'B0BP9Z159S'],
 'retrieved_context': ['New 2023 80x100 High Powered Monoculars for Adults high Powered BAK-4 Prism and FMC Lens Monocular Telescope for Smartphone Monoculars for Bird Watching/Wildlife/Hunting/Hiking 【80x100 High Power Monocular】100 mm object diameter monocular, 80 times magnification. Using optical technology, this monocular uses a fully coated lens to ensure excellent light transmittance and image brightness, provide images with high definition, high quality and high color saturation. 【Compact Monocular】Compact and light monocular 

RAG Pipeline With Grounding Context

In [8]:
class RAGUsedContext(BaseModel):
    id: str = Field(description="The id of the item used to answer the question")
    description: str = Field(description="Shortdescription of the item used to answer the question")

class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")
    references: list[RAGUsedContext] = Field(description="list of items used to answer the questions")

In [11]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
-As an output, you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only retturn the ones that are used in the answer.
*Short description (1-2 sentences) of the item based on the description provided in the context.

-The short description should have the name of the item.
-The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.


Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt


def generate_answer(prompt):

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature = 0.0,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "original_output": answer,
        "answer": answer.answer,
        "references": answer.references,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [12]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [13]:
result = rag_pipeline("Can I get some headphones?", qdrant_client, top_k=10)

In [14]:
result

{'original_output': RAGGenerationResponse(answer='Yes, you can get several types of headphones from the available products:\n\n1. TUNEAKE Kids Headphones (ID: B0C142QS8X)\n- Over ear stereo headphones designed for kids\n- Volume limited to 94dB for hearing protection\n- Foldable and adjustable headband for comfort\n- Soft padded ear cups with perforated mesh\n- 3.5mm jack with 120cm cable, compatible with smartphones, tablets, laptops, and more\n- No built-in microphone but can use device mic\n- Lifetime support and easy customer service\n\n2. Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G)\n- Wireless Bluetooth 5.3 open ear sports headphones\n- Premium sound with bone conduction technology\n- Comfortable, secure fit with ergonomic design\n- IP56 sweat resistant, suitable for workouts and outdoor activities\n- Noise-canceling microphone for calls\n- 8-hour battery life with quick charge\n\n3. Wireless Earbuds S23-vine (ID: B0B9FTVL58)\n- Bluetooth 5.3 wireless earbuds with

In [16]:
print(result['answer'])

Yes, you can get several types of headphones from the available products:

1. TUNEAKE Kids Headphones (ID: B0C142QS8X)
- Over ear stereo headphones designed for kids
- Volume limited to 94dB for hearing protection
- Foldable and adjustable headband for comfort
- Soft padded ear cups with perforated mesh
- 3.5mm jack with 120cm cable, compatible with smartphones, tablets, laptops, and more
- No built-in microphone but can use device mic
- Lifetime support and easy customer service

2. Siniffo Upgraded Bone Conduction Headphones (ID: B0BNHVLF7G)
- Wireless Bluetooth 5.3 open ear sports headphones
- Premium sound with bone conduction technology
- Comfortable, secure fit with ergonomic design
- IP56 sweat resistant, suitable for workouts and outdoor activities
- Noise-canceling microphone for calls
- 8-hour battery life with quick charge

3. Wireless Earbuds S23-vine (ID: B0B9FTVL58)
- Bluetooth 5.3 wireless earbuds with microphone
- 37 hours playback time
- Deep bass sound
- IPX7 waterpro